In [ ]:
#ячейка, которая даёт доступ к датасетам соревнований на kaggle (при необходимости надо ещё сделать !pip install kaggle)
import os
os.environ["KAGGLE_USERNAME"]="valeriianizolia"
os.environ['KAGGLE_KEY']='cac25c733910006ce9e1cef86202110f'
!kaggle competitions download -c iad-deep-learning-sentiment

  0% 0.00/8.37M [00:00<?, ?B/s]
100% 8.37M/8.37M [00:00<00:00, 77.3MB/s]
 97% 577M/595M [00:05<00:00, 69.9MB/s]
100% 595M/595M [00:05<00:00, 109MB/s] 
  0% 0.00/2.08M [00:00<?, ?B/s]
100% 2.08M/2.08M [00:00<00:00, 141MB/s]
 96% 65.0M/68.0M [00:00<00:00, 25.0MB/s]
100% 68.0M/68.0M [00:00<00:00, 82.3MB/s]


In [ ]:
!unzip \*.zip

Archive:  y_train.csv.zip
  inflating: y_train.csv             

Archive:  x_test.txt.zip
  inflating: x_test.txt              

Archive:  random_prediction.csv.zip
  inflating: random_prediction.csv   

Archive:  x_train.txt.zip
  inflating: x_train.txt             

4 archives were successfully processed.


In [ ]:
#для очистки памяти
os.remove("x_test.txt.zip")
os.remove("x_train.txt.zip")
os.remove("y_train.csv.zip")
os.remove("random_prediction.csv.zip")

In [ ]:
import pandas as pd
random_prediction = pd.read_csv("random_prediction.csv")
y_train = pd.read_csv("y_train.csv")
x_train = pd.read_fwf('x_train.txt', header=None)
x_train.to_csv("x_train.csv")
x_test = pd.read_fwf('x_test.txt', header=None)
x_test.to_csv("x_test.csv")

In [ ]:
os.remove("x_train.txt")
os.remove("x_test.txt")

In [ ]:
def delete_punctuation(x):
    punctuation = list(string.punctuation)
    return ''.join([a if a not in punctuation + ['\n'] else ' ' for a in x])

In [ ]:
x_train[0]

0          Stuning even for the non-gamer: This sound tra...
1          The best soundtrack ever to anything.: I'm rea...
2          Amazing!: This soundtrack is my favorite music...
3          Excellent Soundtrack: I truly like this soundt...
4          Remember, Pull Your Jaw Off The Floor After He...
                                 ...                        
3599995    Don't do it!!: The high chair looks great when...
3599996    Looks nice, low functionality: I have used thi...
3599997    compact, but hard to clean: We have a small ho...
3599998    what is it saying?: not sure what this book is...
3599999    Makes My Blood Run Red-White-And-Blue: I agree...
Name: 0, Length: 3600000, dtype: object

In [ ]:
import string
# приведение всех слов к нижнему регистру
x_train[0] = x_train[0].apply(lambda x: x.lower())
x_test[0] = x_test[0].apply(lambda x: x.lower())
# удаление пунктуации
x_train[0] = x_train[0].apply(delete_punctuation)
x_test[0] = x_test[0].apply(delete_punctuation)

In [ ]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip

--2019-12-22 19:15:50--  http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip [following]
--2019-12-22 19:15:51--  https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip [following]
--2019-12-22 19:15:51--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Le

In [ ]:
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Bidirectional
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [ ]:
import os, zipfile
file_name = os.path.abspath('./glove.42B.300d.zip') # get full path of files
zip_ref = zipfile.ZipFile(file_name) # create zipfile object
zip_ref.extractall('./') # extract file to dir
zip_ref.close() # close file
os.remove(file_name) # delete zipped file

In [ ]:
import numpy as np
f = open('./glove.42B.300d.txt')

embeddings_index = dict() 
for line in tqdm_notebook(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
os.remove("glove.42B.300d.txt")

In [ ]:
os.remove("random_prediction.csv")

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a comment to use (приблизительная максимальная длина отзывов теперь 20 слов)

list_sentences_train = x_train[0].values
y = y_train["Probability"].values
list_sentences_test = x_test[0].values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
x_train_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test_pad = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
os.remove("x_train.csv")
os.remove("y_train.csv")
os.remove("x_test.csv")

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
emb_mean, emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.005720101, 0.2951066)

In [ ]:
word_index = tokenizer.word_index
unknown_words = set()
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    else: unknown_words.add(word)

In [ ]:
from keras import backend as K

def auc(y_true, y_pred):
  auc=tf.metrics.auc(y_true, y_pred)[1]
  K.get_session().run(tf.local_variables_initializer())
  return auc

In [ ]:
del x_train, x_test, y_train

In [ ]:
import tensorflow as tf
input_layer = Input((maxlen,), name = 'reviews')
embedding_layer = Embedding(max_features, embed_size, input_length=maxlen, 
                            weights=[embedding_matrix], 
                            trainable = False)(input_layer)
x = Bidirectional(GRU(128, return_sequences=True))(embedding_layer)
x = Dropout(0.3)(x)
x = Bidirectional(GRU(128, return_sequences=False))(x)
x = Dense(64, activation="relu")(x)
output_layer = Dense(1, activation="sigmoid")(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(clipvalue=1, clipnorm=1),
                  metrics=[auc])
print(model.summary())

def schedule(ind):
    a = [0.001, 0.001, 0.0001]
    return a[ind]

lr = LearningRateScheduler(schedule)
    
early_stop = EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1,
                           min_delta=1e-4)


history = model.fit(x_train_pad, y, batch_size=256, epochs = 3, 
                    validation_split = 0.1, verbose = 1, callbacks = [lr])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reviews (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 300)          30000000  
_________________________________________________________________
bidirectional_7 (Bidirection (None, 150, 256)          329472    
_________________________________________________________________
dropout_4 (Dropout)          (None, 150, 256)          0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 256)               295680    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65  

KeyboardInterrupt: ignored

In [ ]:
ind = np.arange(1, x_test_pad.shape[0]+1)

In [ ]:
predicted_probs = model.predict(x_test_pad,batch_size = 1024)

In [ ]:
a = np.arange(1, x_test_pad.shape[0]+1)
predictions = pd.DataFrame(a, columns = ['Id'])
predictions["Probability"]= predicted_probs

In [ ]:
predictions.to_csv("submission_full.csv", index=False)